# Fourier vs cosine transform
One of the problems (or features) of the fourier transform is that it treats the edges of data as if they were repeated infinitely. This can cause problems, for example when blurring, where having a bright edge on one side of the image shouldn't affect the blurring on the other. On the other hand, the cosine transform treats the edges of data as being symmetric. Can we use the cosine transform for fast, effecient, artefact free gaussian blurring?

# Setup

In [60]:
import numpy as np
# on my mac anaconda 3.5 has cv2 - (although I thought that wasn't supported)
import cv2
import scipy.misc
import matplotlib.pyplot as plt
from functools import reduce
from scipy.fftpack import dct, idct
from PIL import Image
%matplotlib inline
plt.rcParams['image.cmap'] = 'gray'

In [84]:
src = scipy.misc.face()[..., 0]
src_h, src_w = src.shape
src = (src * np.indices(src.shape)[1] / src.shape[1]).astype(np.uint8)
plt.imshow(src)
Image.fromarray(src).save('src.png')

In [85]:
def gaussian_quadrant(shape, sds):
    """
    Return a gaussian of the given shape, with the given standard deviations.
    Treats 0,0 as origin
    """
    return reduce(np.multiply, 
                  (np.exp(-dx**2 / (2*sd**2)) 
                   for sd, dx in zip(sds, np.indices(shape))))

def mirror_image(im):
    """
    Takes an image A, and mirrors in x and y
    """
    out = im
    left = np.vstack([im, im[::-1, :]])
    return np.hstack([left, left[:, ::-1]])

def gaussian_blur_cv2_dft(im, amount):
    # gaussian blur using dft
    h, w = im.shape
    quarter_gauss = gaussian_quadrant([h//2, w//2], 
                                      [h/(2*np.pi*amount), w/(2*np.pi*amount)])
    full_gauss = mirror_image(quarter_gauss)
    #print(full_gauss.shape)
    Fsrc = cv2.dft(im.astype(float), None, cv2.DFT_COMPLEX_OUTPUT)
    Fsrc[..., 0] *= full_gauss
    Fsrc[..., 1] *= full_gauss
    inv = cv2.dft(Fsrc, None, cv2.DFT_INVERSE | cv2.DFT_REAL_OUTPUT | cv2.DFT_SCALE)
    return inv.astype(im.dtype)

def gaussian_blur_cv2_dct(im, amount):
    # gaussian blur using cv2 dct
    h, w = im.shape
    gaussian = gaussian_quadrant([h, w], 
                                 [h/(np.pi*amount), w/(np.pi*amount)])
    Fsrc = cv2.dct(im.astype(float))
    Fsrc *= gaussian
    #Fsrc[..., 1] *= full_gauss
    inv = cv2.dct(Fsrc, None, cv2.DCT_INVERSE)
    return inv.astype(im.dtype)

def gaussian_blur_scipy_dct(im, amount, dct_type=2):
    # gaussian blur using scipy dctcv2 dct
    h, w = im.shape
    gaussian = gaussian_quadrant([h, w], 
                                 [h/(np.pi*amount), w/(np.pi*amount)])
    # dct is 1D only, so have to apply to both axes
    Fsrc = dct(dct(src, axis=0, type=dct_type), axis=1, type=dct_type)
    Fsrc *= gaussian
    #Fsrc[..., 1] *= full_gauss
    inv = idct(idct(Fsrc, axis=1, type=dct_type), axis=0, type=dct_type)
    #return inv.astype(im.dtype)
    return (inv/w/h/2/2).astype(im.dtype)

# The Problem
Notice how in the blurred image below there are bright areas on the left hand side.

In [87]:
blur_amount = 50
dft_blur = gaussian_blur(src, blur_amount)
plt.imshow(dft_blur)    
Image.fromarray(dft_blur*2).save('dft_blur.png')

# Solution #1
More data.

If we mirror the src, blur, then crop, we can solve the problem. But this uses 4x the data!

In [98]:
src_h, src_w = src.shape
src_x4 = mirror_image(src)
dft_x4_blur = gaussian_blur_cv2_dft(src_x4, blur_amount)
dft_x4_blur_cropped = dft_x4_blur[:src_h, :src_w]
plt.imshow(dft_x4_blur_cropped)
#Image.fromarray(dft_x4_blur_cropped*2).save('dft_x4_blur_cropped.png')

# Try #2 - opencv dct
Cosine transform instead.

See [wikipedia][1] page on dcts... looks like we need a dct1 to perform proper convolution? But this does appear to work!

[1]:https://en.wikipedia.org/wiki/Symmetric_convolution


In [101]:
dct_blur = gaussian_blur_cv2_dct(src, blur_amount)
plt.imshow(dct_blur) 
Image.fromarray(dct_blur*2).save('dct_blur.png')

In [44]:
dct_x4_blur = gaussian_blur_dct(src_x4, blur_amount)
dct_x4_blur_cropped = dct_x4_blur[:src_h, :src_w]
plt.imshow(dct_x4_blur_cropped)

In [45]:
src.mean(), dct_blur.mean(), dft_blur.mean(), dft_x4_blur_cropped.mean(), dct_x4_blur_cropped.mean()

In [46]:
plt.imshow(dct_x4_blur_cropped.astype(float) - dft_x4_blur_cropped.astype(float), vmin=-2, vmax=2)

# Try #3 - scipy dct
Scipy has `scipy.fftpack.dct` which takes a type of 1,2 or 3. With the default (type 2) we don't see the same artefacts as with the opncv dct. Also, the returned image is scaled incorrectly...

So why does scipy dct type 2 work, but open CV doesn't? 
We get correct scale by dividing `idct(dct(x))` by `width * 2 * height * 2`

From docs, open cv dct uses 
... cos(π(2k+1)j / 2N)

scipy uses
cos(pi*k*(2n+1)/(2N))

In [47]:
dct_blur = gaussian_blur_scipy_dct(src, blur_amount, 2)
plt.imshow(dct_blur) 

## opencv vs scipy dct comparison

In [48]:
# 1d case, 1x32
row_25 = src[25][:32]
assert row_25.shape == (32,)
print("Row 25 mean = {}".format(row_25.mean()))
idct(dct(row_25.astype(float))).mean()/1024/2

In [49]:
plt.plot(dct(row_25, norm='ortho'))
plt.plot(cv2.dct(row_25.astype(float)))

So this looks identical in the 1-d case. What about a 32x2 2d case?

In [50]:
def scipy_2d_dct(im, dct_type=2):
    return dct(dct(im, axis=0, type=dct_type, norm='ortho'), axis=1, type=dct_type, norm='ortho')
    
srcim = src[25:27][..., :32]
assert srcim.shape == (2, 32)

In [51]:
plt.plot(scipy_2d_dct(srcim)[0])
plt.plot(cv2.dct(srcim.astype(float))[0])
plt.figure()
plt.plot(scipy_2d_dct(srcim)[1])
plt.plot(cv2.dct(srcim.astype(float))[1])


# Speed comparisons

In [52]:
%%timeit
dft_blur = gaussian_blur(src, blur_amount)

In [53]:
%%timeit
src_x4 = mirror_image(src)
dft_x4_blur = gaussian_blur(src_x4, blur_amount)
dft_x4_blur_cropped = dft_x4_blur[:src_h, :src_w]

In [54]:
%%timeit
dct_blur = gaussian_blur_dct(src, blur_amount)

In [55]:
%%timeit
dct_blur = gaussian_blur_scipy_dct(src, blur_amount, 2)

# Some images for the blog

In [91]:
def tile(im):
    h,w = im.shape
    ret = np.zeros((h*2, w*2), dtype=im.dtype)
    for x in [0, w]:
        for y in [0,h]:
            ret[y:y+h,x:x+w] = im
    return ret
Image.fromarray(tile(src)).save('src_tiled.png')

In [92]:
Image.fromarray(src_x4).save('src_mirrored.png')